In [1]:
###############################################################################
# Author: Carlos Bobed
# Date: Nov 2020
# Comments: Code to partition a transaction database according to a 
# clustering using its embedding representations
# Modifications:
###############################################################################

import gensim, logging, os, sys, gzip
import time

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',filename='word2vec.out', level=logging.INFO)
                        
## method to read for the Vreeken's codetable format 
## we don't need it to be a generator 
## we do label each code and honor the order in the code table (length, support, lexicographical)
## following Pierre's suggestion, we keep track of the codes and the transaction IDs 
def read_codetable(filename, load_all): 
    codes = {}
    label = 0 
    with open(filename, mode='rt', encoding='UTF-8') as file: 
        for line in file: 
            item_line = list(filter(None, line.rstrip('\n').split(' ')))
            ## only_used => those codes whose usage is > 0
            ## we get the last token, check whether it ends with )
            ## then, we get exactly the contents and check whether the first 
            ## component is different from 0
            if (item_line[-1].endswith(')')): 
                usage,support = item_line[-1][1:-1].split(',')
                if (load_all or int(usage) != 0):
                    codes[label]={'code': item_line[:-1], 'usage':int(usage), 'support':int(support)}
                    label+=1
    return codes        

## to keep track of the transactions id in the database, we read them in a different method
def read_database_db (filename): 
    transactions = {}
    label = 0
    with open(filename, mode='rt', encoding='UTF-8') as file: 
        for line in open(filename, mode='rt', encoding='UTF-8'): 
            if (line.split(':')[0].isnumeric()): 
                aux = line.split(':')[1].rstrip('\n')
                words = filter(None,aux.split(' '))
                transactions[label] = list(words)
                label+=1
    return transactions

def read_database_dat(filename): 
    transactions = {}
    label = 0
    with open(filename, mode='rt', encoding='UTF-8') as file: 
        for line in open(filename, mode='rt', encoding='UTF-8'): 
            aux = line.rstrip('\n')
            words = filter(None,aux.split(' '))
            transactions[label] = list(words)
            label+=1
    return transactions

## read information from the analysis to get back to the .dat file and select them
## we can cluster them according just to the items, or to the transactions themselves

def read_analysis_table (filename): 
    table = {}
    with open(filename, mode='rt', encoding='UTF-8') as file: 
        # skip the first  lines
        for i in range(15): 
            file.readline()
        current_line = file.readline()
        while (current_line != "\n"): 
            aux = current_line.split()[0].split('=>')
            table[int(aux[0])] = int(aux[1])
            current_line = file.readline()
    return table

## convert a transaction in Vreken to the original item name
def convert_transaction_db_to_dat (transaction, table): 
    return sorted([table[int(item)] for item in transaction])

def convert_database_db_to_dat(database, table, output_filename): 
    with open(output_filename, mode='wt', encoding='UTF-8') as file: 
        for i in database: 
            for item in convert_transaction_db_to_dat(database[i], table): 
                file.write(f'{item} ')
            file.write('\n')

## split a database regarding a cluster of items that has been calculated 
## either at item embedding or at transaction embedding level
## for the time being everything is done in memory 
## TODO: process each line at a time
def split_database_items(database, database_name, clusters): 
    ## we create an in-memory database for each cluster
    ## the items currently are stored in the DB as strings as they are tokens for the 
    ## word embedding
    ## the clusters of items can be just sets of integers for speed up reasons
    k=len(clusters)
    in_mem_splitting = {label:[] for label in clusters}
    for i in database: 
        aux_set = set()
        [aux_set.add(int(item)) for item in database[i]]
        ## NOT THE MOST EFFICIENT WAY TO DO IT!!!! 
        ## to speed up this: we should create an inverted index
        [in_mem_splitting[label].append(database[i]) for label in clusters if len(aux_set.intersection(clusters[label])) != 0]
    
    for label in clusters: 
        with open(os.path.join('databases', database_name[:-4]+'_'+label+'_k'+str(k)+'.dat'), mode='wt', encoding='UTF-8') as file: 
            for trans in in_mem_splitting[label]: 
                [file.write(f'{item} ') for item in trans]
                file.write('\n')
                
def split_database_transactions (database_name, clusters): 
    ## the splitting has been already done, and they just give clusters of transactions
    k=len(clusters)
    for label in clusters: 
        with open(os.path.join('databases', database_name[:-4]+'_'+str(label)+'_k'+str(k)+'.dat'), mode='wt', encoding='UTF-8') as file: 
            for trans in clusters[label]: 
                [file.write(f'{item} ') for item in trans]
                file.write('\n')
                
def split_database_transactions_translating (database_name, clusters, table): 
    ## the splitting has been already done, and they just give clusters of transactions
    k=len(clusters)
    for label in clusters: 
        with open(os.path.join('databases', database_name[:-4]+'_'+str(label)+'_k'+str(k)+'.dat'), mode='wt', encoding='UTF-8') as file: 
            for trans in clusters[label]: 
                [file.write(f'{table[int(item)]} ') for item in trans]
                file.write('\n')

C:\Users\cbobed\Anaconda3\envs\rdf2vecEnv\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
dir_name='databases'
database_name = 'adult'
database_filename = database_name + '.db'
database_analysis_filename = database_filename + '.analysis.txt'
database_model_filename = database_filename + '.vect'
codetable_filename = database_name+'-latest-SLIM.ct'

In [3]:
# ## convert all the .db files into .dat 
# for db in [filename for filename in os.listdir('databases') if filename.endswith('.db')]:
#     trans_table = read_analysis_table(os.path.join('databases', db + '.analysis.txt'))
#     db_database = read_database_db(os.path.join('databases', db))
#     convert_database_db_to_dat(db_database, trans_table, os.path.join('databases', db[:-3]+'.dat')) 

In [4]:
### Clustering functions and options 

from gensim.models import Word2Vec
import numpy as np
from sklearn import preprocessing
import math 
import os

model = Word2Vec.load(os.path.join(dir_name, database_model_filename))
labelled_vects = {int(x): model.wv[x] for x in model.wv.vocab}

def calculate_centroids (model, labelled_transactions): 

#     initial code
#     centroids = []
#     for transaction in transaction_list: 
#         words = [model.wv[it] for it in transaction]
#         centroids.append(np.mean(words, axis=0))
#     return centroids

    # more pythonic way
    return { label: np.mean([model.wv[it] for it in  labelled_transactions[label]], axis=0) for label in labelled_transactions}


def calculate_normalized_centroids(model, labelled_transactions): 
    dim = model.wv[labelled_transactions[0][0]].shape[0]
    return { label: preprocessing.normalize(np.mean([model.wv[it] for it in labelled_transactions[label]], axis=0).reshape(1,dim), norm='l2').reshape(dim,) 
                                            for label in labelled_transactions}

### NOTES_ 
### using the overall probability of each item as weight to calculate the centroid 
### lead to decisions which partition the codes further (in the histograms I've observed that 
### the freq of lengths of the codes gets higher in the shorter regions)
### going for a soft clustering technique (considering fuzzy k-means, or hdbscan)

def calculate_weighted_centroids(model, labelled_transactions): 
    # model.wv.vocab['1'].count
    # I was going to get the softmax derived weights but it would add some dependency that we don't want 
    # to explore right now
    # for the time being, let's focus on the global probability of each item being part of a transaction
    
#     total_sum = sum([model.wv.vocab[it].count for it in model.wv.vocab])
#     e_values = {it: math.pow(math.e, model.wv.vocab[it].count / total_sum) for it in model.wv.vocab}
#     total_e_values = sum([evalues[it] for it in e_values])
#     weights = {it:e_values[it] / total_e_values for it in e_values}
    
    result = {}
    weights = {it: model.wv.vocab[it].count/len(labelled_transactions) for it in model.wv.vocab}
    for label in labelled_transactions: 
        item_vect = []
        weight_vect = []
        for it in labelled_transactions[label]: 
            item_vect.append(model.wv[it])
            weight_vect.append(weights[it])
        result[label] = np.average(item_vect, weights=np.array(weight_vect, dtype='float32'), axis=0)
    return result

import time
start = time.time()

if (database_filename.endswith('.db')): 
    database_transactions = read_database_db(os.path.join('.', dir_name, database_filename))
elif (database_filename.endswith('.dat')): 
    database_transactions = read_database_dat(os.path.join('.', dir_name, database_filename))
centroids = calculate_centroids(model, database_transactions)
weighted_centroids = calculate_weighted_centroids(model, database_transactions)
normalized_centroids = calculate_normalized_centroids(model, database_transactions)
end = time.time()

print(end - start)

FileNotFoundError: [Errno 2] No such file or directory: 'databases\\adult.db.vect'

In [20]:
print(centroids[0].shape)
print(weighted_centroids[0].shape)
print(normalized_centroids[0].shape)

(200,)
(200,)
(200,)


In [12]:
## I was going to use scikit directly, but I saw this post
## kudos for him https://towardsdatascience.com/k-means-8x-faster-27x-lower-error-than-scikit-learns-in-25-lines-eaedc7a3a0c8

import faiss
import numpy as np



start = time.time()
## we compute the kmeans of the transactions
database_as_nparray = np.array([centroids[c] for c in centroids])
trans_kmeans = faiss.Kmeans(d=database_as_nparray.shape[1], k=8, niter = 2000, nredo=10)
trans_kmeans.train(database_as_nparray)

end = time.time()
print(end - start)

3.820908546447754


In [13]:
## I was going to use scikit directly, but I saw this post
## kudos for him https://towardsdatascience.com/k-means-8x-faster-27x-lower-error-than-scikit-learns-in-25-lines-eaedc7a3a0c8

#normalizing the vectors, we will have the spherical k-means

import faiss
import numpy as np
from sklearn import preprocessing

start = time.time()
## we compute the kmeans of the transactions
database_as_nparray = np.array([normalized_centroids[c] for c in normalized_centroids])
trans_kmeans = faiss.Kmeans(d=database_as_nparray.shape[1], k=8, niter = 2000, nredo=10)
trans_kmeans.train(database_as_nparray)

end = time.time()
print(end - start)

KeyboardInterrupt: 

In [ ]:
### Might be too far expensive for our purposes

import hdbscan
import time 
start = time.time() 

trans_hdbscan = hdbscan.HDBSCAN(min_cluster_size=1000, prediction_data=True).fit(database_as_nparray)

end = time.time()
print(end - start)

In [8]:
from sklearn_extensions import fuzzy_kmeans as ske
import time 

start = time.time() 

trans_fuzzy_k_means = ske.FuzzyKMeans(k=8, max_iter=300)
trans_fuzzy_k_means.fit(database_as_nparray)
trans_fuzzy_k_means.predict(database_as_nparray)
end = time.time()
print(end - start)

ModuleNotFoundError: No module named 'sklearn.datasets.samples_generator'

In [ ]:
ordered = np.transpose([centroids[c] for c in sorted(centroids)])
np.array_equal(ordered[:,1500], centroids[1500])
ordered[:,1500].shape

In [ ]:
import skfuzzy as fuzz

start = time.time() 
f
# as we don't have the predict feature, we have to keep track of the order 
database_as_np_array_fuzzy = np.transpose(np.array([centroids[c] for c in sorted(centroids)]))

cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans( database_as_np_array_fuzzy, 8, 1.5, error=0.005, maxiter=1000, init=None)
end = time.time()
print(end - start)


In [153]:
idxs = np.argsort(u,axis=0)
print(idxs.shape)
max_values = [ u[idxs[:,idx][-1], idx] for idx in range(idxs.shape[1])  ]
second_max_values = [ u[idxs[:,idx][-2], idx] for idx in range(idxs.shape[1])  ]

count=0
for i in max_values: 
    if i>0.5: 
        count += 1
print(count)


count=0
for i in second_max_values: 
    if i>0.25: 
        count += 1
print(count)

(8, 67557)
20288
176


In [154]:
## we add the fuzzy k-means tagging 
## beware we have made sure of the ordering and it comes transposed [:,i] is the array of k grades of the i-th transaction

def fuzzy_clustering_top_k(database, trans_vects, top_k, fuzzy_grades):
    fuzzy_clusters = {}
    for i in range(fuzzy_grades.shape[0]): 
        fuzzy_clusters[i] = []

    for c in sorted(trans_vects): 
        idx = np.argsort(fuzzy_grades[:,c], axis=0)
        for i in range(-1, -1 -top_k, -1): 
            fuzzy_clusters[idx[i]].append(database[c])
    return fuzzy_clusters

def fuzzy_clustering_threshold(database, trans_vects, threshold, fuzzy_grades):
    fuzzy_clusters = {}
    for i in range(fuzzy_grades.shape[0]): 
        fuzzy_clusters[i] = []

    for c in sorted(trans_vects): 
        idx = np.argsort(fuzzy_grades[:,c], axis=0)
        fuzzy_clusters[idx[-1]].append(database[c])
        if fuzzy_grades[:,c][idx[-2]] >= threshold: 
            fuzzy_clusters[idx[-2]].append(database[c])
    return fuzzy_clusters

f1 = fuzzy_clustering_top_k(database_transactions, centroids, 1, u)
f2 = fuzzy_clustering_top_k(database_transactions, centroids, 2, u)
f4 = fuzzy_clustering_top_k(database_transactions, centroids, 4, u)

f020 = fuzzy_clustering_threshold(database_transactions, centroids, 0.20, u)
f025 = fuzzy_clustering_threshold(database_transactions, centroids, 0.25, u)
f030 = fuzzy_clustering_threshold(database_transactions, centroids, 0.30, u)

In [155]:
for i in f1: 
    print(f'{len(f1[i])} -- {len(f2[i])} -- {len(f4[i])} || {len(f030[i])} -- {len(f025[i])} -- {len(f020[i])}')

10743 -- 11505 -- 11537 || 10743 -- 10756 -- 10847
108 -- 11490 -- 63253 || 108 -- 108 -- 4239
151 -- 12520 -- 64495 || 151 -- 151 -- 3186
10557 -- 14117 -- 14177 || 10562 -- 10659 -- 10928
11188 -- 12144 -- 12168 || 11188 -- 11205 -- 11304
12103 -- 33294 -- 48997 || 12103 -- 12103 -- 12115
11122 -- 26664 -- 42119 || 11122 -- 11122 -- 11146
11585 -- 13380 -- 13482 || 11592 -- 11629 -- 11795


In [64]:
u[:,5]

array([0.12075465, 0.50925817, 0.06249488, 0.02634021, 0.09281569,
       0.05153289, 0.06259125, 0.07421227])

In [8]:
items_as_nparray = np.array([labelled_vects[label] for label in labelled_vects])

items_kmeans = faiss.Kmeans(d=items_as_nparray.shape[1], k=8, niter = 300, nredo=10)
items_kmeans.train(items_as_nparray)

# ## we compute the clustering of the database according to the kmeans of the transaction
# items_D, items_I = trans_kmeans.index.search(items_as_nparray, 1)

777.3660278320312

In [56]:
np.array_equal(centroids[0], centroids[0].reshape(1,200)[0])

True

In [50]:
## we can have then 3 different partitions of the database: 
## clustered according to the items k-means
## clustered according to the items obtained from the "flattened" transaction k-means
## clustered directly from the transaction k-means

## we build the clusters accordingly from the calculated k-means of their vectors 
## done in this way for readability purposes, we can speed up this process by 
## sharing orders 
def item_labelling_from_item_vects (item_vects, items_kmeans): 
    #labelled_vects = {int(x): model.wv[x] for x in model.wv.vocab}
    cluster = {}
    base_label_item = 'item_clust_'
    for i in item_vects: 
        item_D, item_I = items_kmeans.index.search(item_vects[i].reshape(1,200), 1)
        current_label = base_label_item+str(item_I[0,0])
        if current_label not in cluster: 
            cluster[current_label] = set()
        cluster[current_label].add(i)
    return cluster

## we build the clusters accordingly from the items in the transactions belonging to a 
## cluster
def item_labelling_from_trans_vects (database, trans_vects, trans_kmeans): 
# from read_database_*
#     transactions[label] = list(words)
#     label+=1
    
    cluster = {}
    base_label_trans_item = 'trans_item_clust_'
    for t in trans_vects: 
        trans_D, trans_I = trans_kmeans.index.search(trans_vects[t].reshape(1,200), 1)
        current_label = base_label_trans_item+str(trans_I[0,0])
        if current_label not in cluster: 
            cluster[current_label] = set()
        ## centroids/trans_vects share the same ids as database 
        for item in database[t]: 
            cluster[current_label].add(int(item))
    return cluster

## finally, we build the clusters accordingly from the transaction vectors 
def trans_labelling_from_trans_vects (database, trans_vects, trans_kmeans): 
    cluster = {}
    base_label_trans_item = 'trans_clust_'
    for t in trans_vects: 
        trans_D, trans_I = trans_kmeans.index.search(trans_vects[t].reshape(1,200), 1)
        current_label = base_label_trans_item+str(trans_I[0,0])
        if current_label not in cluster: 
            cluster[current_label] = []
        ## centroids/trans_vects share the same ids as database 
        cluster[current_label].append(database[t])
    return cluster

def trans_labelling_from_trans_vects_normalizing (database, trans_vects, trans_kmeans): 
    cluster = {}
    base_label_trans_item = 'trans_clust_'
    for t in trans_vects: 
        trans_D, trans_I = trans_kmeans.index.search(preprocessing.normalize(trans_vects[t].reshape(1,200)), 1)
        current_label = base_label_trans_item+str(trans_I[0,0])
        if current_label not in cluster: 
            cluster[current_label] = []
        ## centroids/trans_vects share the same ids as database 
        cluster[current_label].append(database[t])
    return cluster

import random
## We randomly split transaction database 
def trans_labelling_random(database, k): 
    cluster={}
    ## beware: database is a dict, not a list 
    idx_list = [i for i in range (len(database))]
    size = int(math.ceil(len(idx_list)/k))
    print(f'size of shuffle: {len(idx_list)}')
    print(f'Partitioning DB of {len(idx_list)} in {k} sets of {size} elements')
    base_label='rand_clust_'
    random.shuffle(idx_list)
    shuffled = [idx_list[i::k] for i in range(k)]
    print(f'size of shuffle: {len(shuffled)}')
    for (i,vect) in enumerate([ idx_list[i::k] for i in range(k)]):
        current_label = base_label+str(i)
        if current_label not in cluster: 
            cluster[current_label] = []
        for j in vect: 
            cluster[current_label].append(database[j])
    for cl in enumerate(cluster): 
        print(f'{cl[0]} - {cl[1]} -- len {len(cluster[cl[1]])}')
    return cluster



In [38]:
# import random 

# l = [i for i in range(100)]

# def partition (list_in, n):
#     random.shuffle(list_in)
#     return [list_in[i::n] for i in range(n)]

# partition(l,2)

In [39]:
# item_cluster = item_labelling_from_item_vects(labelled_vects, items_kmeans)
# trans_item_cluster = item_labelling_from_trans_vects(database_transactions, centroids, trans_kmeans)
trans_cluster = trans_labelling_from_trans_vects(database_transactions, centroids, trans_kmeans)

In [54]:
local_database_filename=database_filename[:-3]+'_ord_200d_k8.dat'
print(database_filename)
print(local_database_filename)
translation_table_test = read_analysis_table(os.path.join('databases',  database_analysis_filename))

# split_database_items(database_transactions, database_filename, item_cluster)
# split_database_items(database_transactions, database_filename, trans_item_cluster)
split_database_transactions("test_"+local_database_filename, trans_cluster)
split_database_transactions_translating(local_database_filename, trans_cluster, translation_table_test)

chessBig.db
chessBig_ord_200d_k8.dat


KeyError: 64

In [67]:
## Random partitioning
local_database_filename=database_filename[:-3]+'_rand_k8.dat'
print(database_filename)
print(local_database_filename)
translation_table_test = read_analysis_table(os.path.join('databases',  database_analysis_filename))
rand_cluster = trans_labelling_random(database_transactions, 8)
split_database_transactions_translating(local_database_filename, trans_cluster, translation_table_test)

adult.db
adult_rand_k8.dat
size of shuffle: 48842
Partitioning DB of 48842 in 8 sets of 6106 elements
size of shuffle: 8
0 - rand_clust_0 -- len 6106
1 - rand_clust_1 -- len 6106
2 - rand_clust_2 -- len 6105
3 - rand_clust_3 -- len 6105
4 - rand_clust_4 -- len 6105
5 - rand_clust_5 -- len 6105
6 - rand_clust_6 -- len 6105
7 - rand_clust_7 -- len 6105


KeyError: 108

As expected, if the items are not sparsely distributed the item-base clustering seems easily go to complete databases (if you have an item that's very frequent it's going to "infect" all of the transactions). 

Below is just testing code

In [66]:
translation_table_test = read_analysis_table(os.path.join('databases',  database_analysis_filename))
print(translation_table_test)

{0: 10, 1: 13, 2: 15, 3: 1, 4: 19, 5: 18, 6: 17, 7: 14, 8: 11, 9: 16, 10: 12, 11: 2, 12: 6, 13: 5, 14: 9, 15: 3, 16: 7, 17: 8, 18: 4}


In [59]:
dat_database_test = read_database_dat(os.path.join('databases', database_filename_dat))

FileNotFoundError: [Errno 2] No such file or directory: 'databases/iris.dat'

In [67]:
db_database_test = read_database(os.path.join('databases', database_filename))
convert_database_db_to_dat(db_database_test, translation_table_test , os.path.join('databases', 'testing-test.dat') )

In [84]:
cluster={'primer':set(range(18)), 'second':set(range(18,19))}

In [85]:
cluster

{'primer': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17},
 'second': {18}}

In [86]:
split_database_items(db_database_test, 'test-test', cluster)

[18, 1]
[150, 2]


In [95]:
cluster={'primer':[db_database_test[i] for i in range(80)], 'second':[db_database_test[i] for i in range(80, len(db_database_test))]}

In [96]:
split_database_transactions('test-test', cluster)

In [97]:
len(db_database_test)

150